In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Read the data
data = pd.read_csv('../../../.Dataset/BTC-USD-3.2018-3.2024.csv')
data.head()

In [ ]:
# Set the date as the index
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
data.head()

In [ ]:
# Feature Engineering
def createFeatures(df):
    df = pd.DataFrame(df)

    
    # df['Close_Diff'] = df['Adj Close'].diff()
        
    # Moving averages - different periods
    df['MA200'] = df['Close'].rolling(window=200).mean() 
    df['MA100'] = df['Close'].rolling(window=100).mean() 
    df['MA50'] = df['Close'].rolling(window=50).mean() 
    df['MA26'] = df['Close'].rolling(window=26).mean() 
    df['MA20'] = df['Close'].rolling(window=20).mean() 
    df['MA12'] = df['Close'].rolling(window=12).mean() 
    
    # SMA Differences - different periods
    df['DIFF-MA200-MA50'] = df['MA200'] - df['MA50']
    df['DIFF-MA200-MA100'] = df['MA200'] - df['MA100']
    df['DIFF-MA200-CLOSE'] = df['MA200'] - df['Close']
    df['DIFF-MA100-CLOSE'] = df['MA100'] - df['Close']
    df['DIFF-MA50-CLOSE'] = df['MA50'] - df['Close']
    
    # Moving Averages on high, lows, and std - different periods
    # df['MA200_low'] = df['Low'].rolling(window=200).min()
    # df['MA14_low'] = df['Low'].rolling(window=14).min()
    # df['MA200_high'] = df['High'].rolling(window=200).max()
    # df['MA14_high'] = df['High'].rolling(window=14).max()
    df['MA20dSTD'] = df['Close'].rolling(window=20).std() 
    
    # Exponential Moving Averages (EMAS) - different periods
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['EMA100'] = df['Close'].ewm(span=100, adjust=False).mean()
    df['EMA200'] = df['Close'].ewm(span=200, adjust=False).mean()

    # Shifts (one day before and two days before)
    df['close_shift-1'] = df.shift(-1)['Close']
    df['close_shift-2'] = df.shift(-2)['Close']

    # Bollinger Bands
    df['Bollinger_Upper'] = df['MA20'] + (df['MA20dSTD'] * 2)
    df['Bollinger_Lower'] = df['MA20'] - (df['MA20dSTD'] * 2)
    df['Bollinger_Middle'] = df['MA20'] 
    
    # Relative Strength Index (RSI)
    # df['K-ratio'] = 100*((df['Close'] - df['MA14_low']) / (df['MA14_high'] - df['MA14_low']) )
    # Relative Strength Index (RSI) not using the K-ratio
    # df['RSI'] = df['K-ratio'].rolling(window=3).mean() 

    # Moving Average Convergence/Divergence (MACD)
    df['MACD'] = df['EMA12'] - df['EMA26']
    
    # Replace nas 
    nareplace = df.at[df.index.max(), 'Close']    
    df.fillna((nareplace), inplace=True)
    
    # 
    return df

In [ ]:
# List of considered Features
FEATURES = [
            # 'High',
            # 'Low',
            # 'Open',
            'Close',
            # 'Volume',
#             'Day',
#             'Month',
#             'Year',
            # 'Adj Close',
#              'close_shift-1',
#              'close_shift-2',
            'MACD',
            # 'RSI',
            # 'MA200',
#             'MA200_high',
#             'MA200_low',
            'Bollinger_Upper',
            'Bollinger_Lower',
            # 'MA100',            
#             'MA50',
            'MA26',
#             'MA14_low',
#             'MA14_high',
            # 'MA12',
            'EMA20',
            # 'EMA100',
#             'EMA200',
#               'DIFF-MA200-MA50',
#               'DIFF-MA200-MA100',
#             'DIFF-MA200-CLOSE',
#             'DIFF-MA100-CLOSE',
#             'DIFF-MA50-CLOSE',
            # 'MA20dSTD',
            # 'Close_Diff',
            # 'K-ratio'
           ]

# Create the dataset with features
df_features = createFeatures(data)

# Shift the timeframe by 10 month -> Start date is 2010-11-01
use_start_date = pd.to_datetime("2018-10-01")
df_features = df_features[df_features.index > use_start_date].copy()

# Filter the data to the list of FEATURES
df_filtered = df_features[FEATURES].copy()

# Filter the data to the list of FEATURES
print(df_filtered.head().to_string())  
    

# Filter the data to the list of FEATURES
df_filtered = df_filtered.dropna()
print(df_filtered.head().to_string())

# Create the lineplot
plt.figure(figsize=(15, 7))
plt.plot(df_filtered)
plt.title('BTC/USD Close price')
plt.grid()
plt.legend(df_filtered.columns)
plt.show()


In [ ]:
# Scale the data
df = df_filtered.copy()

# Transform the data by scaling each feature to a range between 0 and 1
scaler = MinMaxScaler()
np_data = scaler.fit_transform(df)

# Creating a separate scaler that works on a single column for scaling predictions
scaler_pred = MinMaxScaler()
df_close = pd.DataFrame(df['Close'])
np_close = scaler_pred.fit_transform(df_close)

In [ ]:
print(df_filtered.shape)

In [ ]:
# Set the sequence length - this is the timeframe used to make a single prediction
sequence_length = 7

# Training, validation, test
train_size = int(len(np_data) * 0.70)
test_size = len(np_data) - train_size
train_data, test_data = np_data[0:train_size], np_data[train_size:len(np_data)]

In [ ]:
print(train_data.shape, test_data.shape)

In [ ]:
# The RNN needs data with the format of [samples, time steps, features]
# Here, we create N samples, sequence_length time steps per sample, and 6 features
def partition_dataset(data, time_steps):
    x, y = [], []
    data_len = data.shape[0]
    for i in range(data_len - time_steps):
        row = [r for r in data[i: i + time_steps]]
        x.append(row)
        label = data[i + time_steps][0]
        y.append(label)
    return np.array(x), np.array(y)
    

# Generate training data and test data
X_train, y_train = partition_dataset(train_data, sequence_length)
X_test, y_test = partition_dataset(test_data, sequence_length)

In [ ]:
# Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


In [ ]:
# Reshape input to be [samples, time steps, features] which is required for LSTM
x_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(FEATURES))
x_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(FEATURES))

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# Configure the neural network model
model = Sequential()

# Configure the Neural Network Model with n Neurons - inputshape = t Timestamps x f Features
n_neurons = X_train.shape[1] * x_train.shape[2]
print('timesteps: ' + str(x_train.shape[1]) + ',' + ' features:' + str(x_train.shape[2]))
# model.add(GRU(n_neurons, input_shape=(x_train.shape[1], x_train.shape[2]))) 
# model.add(Dense(1))
model.add(SimpleRNN(n_neurons, return_sequences=True,
          input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(SimpleRNN(n_neurons, return_sequences=True))
model.add(SimpleRNN(n_neurons))
model.add(Dense(1))


# Configure the Model   
optimizer='adam'; loss='mean_squared_error'; epochs = 100; batch_size = 64

# uncomment to customize the learning rate
learn_rate = "standard" # 0.05

parameter_list = ['epochs ' + str(epochs), 'batch_size ' + str(batch_size), 'optimizer ' + str(optimizer) + ' with learn rate ' + str(learn_rate), 'loss ' + str(loss)]
print('Parameters: ' + str(parameter_list))

# Compile and Training the model
model.compile(optimizer=optimizer, loss=loss)
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=1)

# Plot training & validation loss values
fig, ax = plt.subplots(figsize=(12, 6), sharex=True)
plt.plot(history.history["loss"])
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc="upper left")
plt.show()

In [ ]:
# Get x_val, x_test for the prediction from the last sequence_length days of the training set and the first sequence_length days of the validation set
x_test_pred = np.concatenate((x_train[-sequence_length:], x_test))
print(x_test_pred.shape)
# Predict the values
y_test_pred = model.predict(x_test_pred)

# Inverse the scaling of the predicted values
y_test_pred_unscaled = scaler_pred.inverse_transform(y_test_pred)


# Create a dataframe for the test set matching sliding window
day_test = pd.to_datetime(df_filtered.index[-len(y_test_pred):])
df_test = pd.DataFrame(day_test, columns=['Date'])
df_test['Y_test_pred'] = y_test_pred_unscaled
df_test['Y_test'] = df_filtered['Close'][-len(y_test_pred):].values
df_test.set_index('Date', inplace=True)


# # Plot df_val and df_test
plt.figure(figsize=(15, 7))
plt.plot(df_test['Y_test_pred'], label='Y_test_pred')
plt.plot(df_test['Y_test'], label='Y_test')
plt.title('BTC/USD Close price')
plt.grid()
plt.legend()
plt.show()


In [ ]:
df_temp = pd.DataFrame(np_data, columns=FEATURES)
# Loop to predict the next day
i = 1
num_days = 90
while i <= num_days:
    # Get the last sequence_length days
    x_pred = df_temp[-sequence_length:]
    x_pred_input = x_pred.values
    x_pred_input = x_pred_input.reshape(1, sequence_length, len(FEATURES))
    print("x_pred_input of day " + str(i) + ": \n" + str(x_pred_input))
    # Predict the value
    y_pred = model.predict(x_pred_input)
    y_pred_unscaled = scaler_pred.inverse_transform(y_pred)
    print("y_pred of day " + str(i) + ": " + str(y_pred_unscaled))
    # unscaling the data df_temp
    df_temp = pd.DataFrame(scaler.inverse_transform(df_temp), columns=FEATURES)
    # Add the predicted value to the df_temp dataframe
    df_temp = pd.concat([df_temp, pd.DataFrame(y_pred_unscaled, columns=['Close'])], ignore_index=True)
    # Create features for the predicted value
    df_temp = createFeatures(df_temp)
    df_temp = df_temp[FEATURES].copy()
    df_temp = df_temp.dropna()
    # Scale the data
    df_temp = pd.DataFrame(scaler.fit_transform(df_temp), columns=FEATURES)

    # Increment the counter
    i += 1
    

In [ ]:
df_forecast_unscale = pd.DataFrame(scaler.inverse_transform(df_temp), columns=FEATURES)
df_forecast_unscale

In [ ]:
# Get the df_temp with the predicted values
df_forecast = df_temp[-num_days:]

# Unscale the data
df_forecast_unscale = pd.DataFrame(scaler.inverse_transform(df_forecast), columns=FEATURES)

# Create dataframe forecast with the predicted values and the dates
day_forecast = pd.to_datetime(df_filtered.index[-1] + pd.DateOffset(1))
df_forecast_unscale['Date'] = pd.date_range(start=day_forecast, periods=num_days, freq='D')
df_forecast_unscale.set_index('Date', inplace=True)



In [ ]:
# Dateframe of Close training set
df_train = df_filtered['Close'][:train_size].copy()

# Plot the forecast for df_val, df_test and df_forecast
plt.figure(figsize=(15, 7))
plt.plot(df_train, label='Train')
plt.plot(df_test['Y_test_pred'], label='Test prediction')
plt.plot(df_test['Y_test'], label='Test')
plt.plot(df_forecast_unscale['Close'], label='Forecast next 30 days')
plt.title('BTC/USD Close price')
plt.grid()
plt.legend()
plt.show()


In [ ]:
# Model Evaluation
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    smape = np.mean((np.abs(y_pred - y_true) * 200 / (np.abs(y_pred) + np.abs(y_true))))
    return rmse, mape, smape

# Calculate the metrics df_test
rmse, mape, smape = calculate_metrics(df_test['Y_test'], df_test['Y_test_pred'])
print('Metrics for Test Data:')
print('RMSE: ', rmse)
print('MAPE: ', mape)
print('SMAPE: ', smape)